# **Credit Card Fraud Detection**

![logo](images/credit-card-fraud-detection.webp)

## Introdução

O objetivo deste projeto é desenvolver um modelo de *Machine Learning* capaz de identificar transações fraudulentas em cartões de crédito. Usando informações disponíveis, o modelo aprenderá a diferenciar as transações legítimas das fraudulentas.

O modelo vai ser desenvolvido utilizando a linguagem *Python* e vai ser testado e treinado com dados de um *dataset* disponível no *Kaggle* [Kaggle](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).

A abordagem realizada seguirá a metodologia *CRISP-DM* (Cross-Industry Standard Process for Data Mining), que inclui as seguintes etapas: compreensão do tema, compreensão dos dados, preparação dos dados, modelagem, avaliação e implantação.

## Estrutura do Projeto

O projeto é composto por:

- code.ipynb: *Juptyer Notebook* com a abordagem realizada e o código desenvolvido;
- images: pasta com as imagens utilizadas no projeto.
- CreditCardFraudDetection: pasta com o *dataset* utilizado no projeto.

## Tecnologias Utilizadas

Como já foi referido, a linguagem de programação utilizada será o *Python* e as bibliotecas utilizadas foram:

- *Matplotlib* e *Seaborn* para visualização dos dados;
- *Pandas* para manipulação dos dados;
- *NumPy* para tratar os dados númericos;
- *Scikit-learn* para desenvolver o modelo de *Machine Learning*.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
import numpy as np

import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
warnings.filterwarnings('ignore')

## Compreensão do Tema
 
Os cartões de crédito são atualmente um dos meios de pagamento mais adotados, especialmente com a expansão do comércio online. Contudo, essa evolução aumentou a exposição dos consumidores a fraudes, sendo a segurança um aspeto essencial tanto para as instituições financeiras quanto para os consumidores.

Este problema tem como objetivo prever se uma transação de cartão de crédito é **legítima** ou **fraudulenta**, utilizando um *dataset*. A necessidade principal é identificar transações suspeitas com precisão, de forma a prevenir impactos negativos financeiros.

O objetivo do modelo consiste em classificar cada transação num dos dois grupos mutuamente exclusivos:
- **Legítima**: Transação autorizada, realizada pelo titular do cartão.
- **Fraudulenta**: Transação não autorizada, efetuada por terceiros.

Para abordar o problema, será desenvolvido um modelo de *machine learning* especializado em classificação binária, capaz de distinguir transações legítimas das fraudulentas com alto grau de precisão. 

O modelo será treinado utilizando uma abordagem de *supervised learning*, com a meta de alcançar uma alta taxa de precisão e recall nos dados de teste, assegurando eficácia prática e capacidade de minimizar falsos positivos e falsos negativos na deteção de fraudes.

## Compreensão dos Dados

Antes de começar a compreender os dados, é necessário importar os vários ficheiros e juntá-los num único *dataset*.

In [ ]:
cities = pd.read_csv('CreditCardTransactions/cities.csv')
customers = pd.read_csv('CreditCardTransactions/customers.csv')
merchants = pd.read_csv('CreditCardTransactions/merchants.csv')
transactions = pd.read_csv('CreditCardTransactions/transactions.csv')

# Merge do customers.csv e cities.csv usando 'city'
customers_cities = pd.merge(customers, cities, on='city', how='left')

# Merge do transactions.csv e customers_cities usando 'cc_num'
transactions_customers = pd.merge(transactions, customers_cities, on='cc_num', how='left')

# Merge final do transactions_customers e merchants.csv usando 'merchant'
final_data = pd.merge(transactions_customers, merchants, on='merchant', how='left')

# Guardar o resultado final
final_data.to_csv('CreditCardTransactions/merged_dataset.csv', index=False)

data = pd.read_csv('CreditCardTransactions/merged_dataset.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  30000 non-null  int64  
 1   trans_date_trans_time  29900 non-null  object 
 2   cc_num                 30000 non-null  int64  
 3   device_os              12036 non-null  object 
 4   merchant               30000 non-null  object 
 5   amt                    29900 non-null  float64
 6   trans_num              30000 non-null  object 
 7   unix_time              30000 non-null  int64  
 8   is_fraud               30000 non-null  int64  
 9   first                  29990 non-null  object 
 10  last                   29990 non-null  object 
 11  gender                 29990 non-null  object 
 12  street                 29990 non-null  object 
 13  city                   29990 non-null  object 
 14  zip                    29784 non-null  float64
 15  jo

O *dataset* é constituído por 30000 entradas e 25 colunas. Cada entrada
representa uma transação de cartão de crédito. As colunas contêm informações sobre a transação, como o valor, a data, a localização, entre outros que vamos ver com detalhe mais à frente.

Considerando que o dataset possui um número razoável de transações, que podem ser divididas em conjuntos de treino e teste, é provável que se consiga extrair conclusões valiosas e fazer previsões precisas. Por isso, não parece necessário criar novos dados.

Além disso, a dimensionalidade parece adequada para o problema em questão e, tendo em conta que os dados parecem ser consistentes e provenientes da mesma fonte, não se espera encontrar problemas significativos de compatibilidade ou coerência.

In [3]:
data.head()

,index,trans_date_trans_time,cc_num,device_os,merchant,amt,trans_num,unix_time,is_fraud,first,last,gender,street,city,zip,job,dob,lat,long,city_pop,state,category,merch_lat,merch_long,merchant_id
0,5381,2023-01-01 00:39:03,2801374844713453,NaN,Merchant_85,252.75,TRANS_662964,1672533543,0,Jane,Smith,F,1st Ave,Chicago,NaN,NaN,2002-10-12,41.8781,-87.6298,2716000.0,IL,NaN,NaN,76.433212,85.0
1,4008,2023-01-01 01:16:08,3460245159749480,NaN,Merchant_23,340.17,TRANS_134939,1672535768,0,Alice,Wilson,M,Broadway,New York,90146.0,Nurse,2001-12-23,40.7128,-74.0060,8419600.0,NY,Entertainment,27.177588,-64.857435,23.0
2,1221,2023-01-01 01:24:28,7308701990157768,macOS,Merchant_70,76.38,TRANS_258923,1672536268,0,Bob,Clark,M,Broadway,Phoenix,36374.0,Doctor,1978-12-13,33.4484,-112.0740,1680992.0,AZ,Electronics,31.730070,-67.777407,70.0
3,9609,2023-01-01 02:06:57,8454886440761098,X11,Merchant_33,368.88,TRANS_226814,1672538817,0,Mike,Brown,F,2nd Ave,Phoenix,34323.0,Teacher,1965-04-21,33.4484,-112.0740,1680992.0,AZ,Electronics,-5.005953,146.873847,33.0
4,5689,2023-01-01 02:10:54,6350332939133843,NaN,Merchant_90,323.32,TRANS_668449,1672539054,0,Mike,Williams,F,Maple St,New York,77743.0,Nurse,1997-05-17,40.7128,-74.0060,8419600.0,NY,Groceries,79.065894,40.668693,90.0


Assim, vamos analisar com mais detalhe os atributos do *dataset*.

| **Atributo**            | **Tipo/Escala**        | **Significado**                                                  |
|--------------------------|-------------------------|-------------------------------------------------------------------|
| `index`                  | Numérico Discreto Ratio       | Identificador único para cada linha do dataset                |
| `trans_date_trans_time`  | Númerico Discreto Intervalo  | Data e hora da transação                           |
| `cc_num`                 | Numérico Discreto Ratio   | Número do cartão de crédito utilizado na transação               |
| `device_os`              | Categórico Nominal     | Sistema operativo do dispositivo utilizado para a transação    |
| `merchant`               | Categórico Nominal     | Nome/Identificador do comerciante                             |
| `amt`                    | Numérico Contínuo Ratio     | Valor da transação (em unidades monetárias)                      |
| `trans_num`              | Numérico Discreto Ratio    | Identificador único para cada transação                          |
| `unix_time`              | Numérico Contínuo Intervalo    | Timestamp Unix que representa a data e hora da transação         |
| `is_fraud`               | Numérico Binário     | Indicador de fraude na transação (`0`: Não fraudulenta, `1`: Fraudulenta) |
| `first` e `last`         | Categórico Nominal     | Primeiro e último nome do titular do cartão                      |
| `gender`                 | Categórico Nominal     | Género do titular do cartão                                       |
| `street`                 | Categórico Nominal     | Nome da rua                                                      |
| `city`                   | Categórico Nominal     | Cidade do cliente                                                |
| `zip`                    | Numérico Contínuo Ratio    | Código postal                                                    |
| `state`                  | Categórico Nominal     | Estado                                                           |
| `job`                    | Categórico Nominal     | Profissão do titular do cartão                                   |
| `dob`                    |   Númerico Discreto Intervalo | Data de nascimento do titular do cartão                          |
| `lat` e `long`           | Numérico Contínuo Ratio     | Latitude e longitude da localização do cliente                   |
| `city_pop`               | Numérico Contínuo Ratio     | População da cidade onde o cliente reside                        |
| `category`               | Categórico Nominal     | Categoria do comerciante                                         |
| `merch_lat` e `merch_long`| Numérico Contínuo Ratio     | Latitude e longitude do comerciante                              |
| `merchant_id`            | Numérico Discreto Ratio   | Identificador único do comerciante                               |

In [6]:
data.describe()

,index,cc_num,amt,unix_time,is_fraud,zip,lat,long,city_pop,merch_lat,merch_long,merchant_id
count,30000.00000,3.000000e+04,29900.000000,3.000000e+04,30000.000000,29784.000000,10020.000000,10020.000000,1.002000e+04,29401.000000,29990.000000,29990.000000
mean,14994.93820,5.638691e+15,250.063287,1.705650e+09,0.019033,58070.908944,35.726876,-98.630250,3.704410e+06,2.990787,-7.727705,50.446215
std,8664.71394,2.743709e+15,144.106058,1.530499e+07,0.136644,24749.348964,4.531306,15.963517,2.323382e+06,55.651821,103.254575,28.939210
min,0.00000,1.001432e+15,1.010000,1.672534e+09,0.000000,10008.000000,29.760400,-118.243700,1.680992e+06,-88.616543,-178.256215,1.000000
25%,7478.75000,3.256119e+15,125.235000,1.696269e+09,0.000000,39192.000000,33.448400,-112.074000,2.328000e+06,-46.105529,-101.993026,25.000000
50%,14999.50000,5.491563e+15,249.625000,1.706376e+09,0.000000,58583.000000,34.052200,-95.369800,2.716000e+06,0.067189,-16.648430,50.000000
75%,22499.25000,8.149117e+15,375.242500,1.718328e+09,0.000000,78251.000000,40.712800,-87.629800,3.979576e+06,49.823343,90.051574,76.000000
max,29999.00000,1.000000e+16,499.970000,1.730124e+09,1.000000,99994.000000,41.878100,-74.006000,8.419600e+06,89.069132,178.663853,100.000000


Com base nos dados estatísticos dos atributos numéricos, é possível extrair algumas informações relevantes.

O valor médio das transações (`amt`) é de 250.06, com um mínimo registrado de 1.01 e um máximo de 499.97. O desvio padrão é de 144.10, indicando uma variabilidade significativa nos valores transacionados. Transações que apresentam valores significativamente acima da média ou próximos ao valor máximo podem ser consideradas suspeitas.

A incidência de fraudes (`is_fraud`) é relativamente baixa, com uma média de 0.019, o que sugere que aproximadamente 1.9% das transações são fraudulentas. Isto revela que o *dataset* não é equilibrado, pois contém poucas transações fraudulentas em comparação com as legítimas.

Observa-se também a presença de dados ausentes nalguns atributos que parecem ter maior importância para o problema. Apenas 10.020 registros, de um total de 30.000, possuem informações de latitude e longitude do titular (`lat` e `long`). Além disso, o atributo `zip` está preenchido em 29.784 registros. A ausência de dados nestes casos pode indicar tentativas de ocultar informações por parte de atacantes, sendo, portanto, um possível indicativo de atividade suspeita.

Em relação ao tempo das transações (`unix_time`), estas ocorrem dentro de um intervalo de timestamps Unix que vai de 1.672.534.000 a 1.730.124.000. Transações realizadas em horários incomuns ou fora do padrão normal de atividade do cliente podem indicar transações que devem ser investigadas.

In [7]:
data.describe(include=['O'])

,trans_date_trans_time,device_os,merchant,trans_num,first,last,gender,street,city,job,dob,state,category
count,29900,12036,30000,30000,29990,29990,29990,29990,29990,29784,29990,10020,29401
unique,29868,5,101,29470,108,108,2,102,6,7,1062,5,5
top,2023-03-19 03:05:24,Windows,Merchant_72,TRANS_600014,Jane,Williams,F,Elm St,Test City,Lawyer,1965-10-17,CA,Groceries
freq,2,3049,339,4,1489,1442,15414,1780,19970,6443,237,2181,7193


Nos atributos categóricos, observa-se uma alta repetição de valores. Por exemplo, o nome "Jane" aparece 1.489 vezes no atributo `first`, e "Williams" é o sobrenome mais comum, com 1.442 ocorrências. Esta repetição associada à repetição de datas de nascimento sugere o possível uso de identidades falsas, o que é um indicador crítico na deteção de fraudes.

A análise dos endereços revela que apenas 102 ruas e 6 cidades únicas estão presentes nos registros. A rua "Elm St" aparece 1.780 vezes, e "Test City" é a cidade mais comum, com 19.970 ocorrências. Essa concentração geográfica pode indicar a operação de um esquema de fraude localizado. Em operações legítimas, espera-se uma distribuição mais ampla de localidades.

Há apenas 101 comerciantes únicos em 30.000 registros, com "Merchant_72" sendo o mais frequente, aparecendo 339 vezes. Uma alta concentração de transações em determinados comerciantes pode sugerir que esses estabelecimentos estão associados a um número maior de fraudes.

Notou-se, também, uma quantidade significativa de dados ausentes, como o sistema operativo do dispositivo (`device_os`) e o estado (`state`). Apenas 12.036 registros possuem informações sobre o sistema operativo, e somente 10.020 registros incluem o estado. A ausência de informações críticas pode ser uma tentativa deliberada de ocultar detalhes essenciais, prática comum em esquemas fraudulentos.